# Explore the data

## 1. Workspace setup

In [ ]:
%load_ext jupyter_black
import json
import pandas as pd
import numpy as np
import seaborn as sns
import open3d as o3d
import glob
import matplotlib.pyplot as plt
from pathlib import Path
from tqdm import tqdm

In [ ]:
sns.set_theme(context="paper")

## 2. Count number of samples

In [ ]:
path = Path("P:\\Projects\\LMB_4Dspine\\back_scan_database")

backscan_dirs = glob.glob(str(path / "**" / "*.ply"))
metadata_dirs = glob.glob(str(path / "**" / "*.json"))

dirs_df = pd.DataFrame({"backscan": backscan_dirs, "metadata": metadata_dirs})
dirs_df["raw"] = dirs_df["backscan"].str.contains("raw")
dirs_df["processed"] = dirs_df["backscan"].str.contains("processed")

dirs_df["balgrist"] = dirs_df["backscan"].str.contains("balgrist")
dirs_df["croatian"] = dirs_df["backscan"].str.contains("croatian")
dirs_df["italian"] = dirs_df["backscan"].str.contains("italian")
dirs_df["ukbb"] = dirs_df["backscan"].str.contains("ukbb")

dirs_df.groupby("raw")[["balgrist", "croatian", "italian", "ukbb"]].sum()

## 3. Load one subject each

In [ ]:
bal_id = "balgrist_1"
cro_id = "croatian_1"
it_id = "italian_10"
ukbb_id = "ukbb_1238"

backscan_bal = o3d.io.read_point_cloud(str(path / bal_id / f"{bal_id}_processed.ply"))
backscan_cro = o3d.io.read_point_cloud(str(path / cro_id / f"{cro_id}_processed.ply"))
backscan_it = o3d.io.read_point_cloud(str(path / it_id / f"{it_id}_processed.ply"))
backscan_ukbb = o3d.io.read_point_cloud(
    str(path / ukbb_id / f"{ukbb_id}_processed.ply")
)

with open(path / bal_id / f"{bal_id}_metadata_processed.json", "r") as f:
    metadata_bal = json.load(f)

with open(path / cro_id / f"{cro_id}_metadata_processed.json", "r") as f:
    metadata_cro = json.load(f)

with open(path / it_id / f"{it_id}_metadata_processed.json", "r") as f:
    metadata_it = json.load(f)

with open(path / ukbb_id / f"{ukbb_id}_metadata_processed.json", "r") as f:
    metadata_ukbb = json.load(f)

## 4. Explore metadata

In [ ]:
metadata_keys = {}

metadata_keys["balgrist"] = metadata_bal.keys()
metadata_keys["croatian"] = metadata_cro.keys()
metadata_keys["italian"] = metadata_it.keys()
metadata_keys["ukbb"] = metadata_ukbb.keys()

metadata_keys

Common keys: id, dataset, age, gender, pcDicomOutput, SpecialPts, esl, isl, status, pipelineSteps

Balgrist, italian, ukbb: height, weight, recordingTime, operations

Balgrist-only: recordingDatetime

In [ ]:
print(metadata_bal["isl"].keys())
print(metadata_cro["isl"].keys())
print(metadata_it["isl"].keys())
print(metadata_ukbb["isl"].keys())

print(metadata_bal["esl"].keys())
print(metadata_cro["esl"].keys())
print(metadata_it["esl"].keys())
print(metadata_ukbb["esl"].keys())

For Balgrist and UKBB, the esl and isl dictionary keys are 'pcdicomApp'. For croatian the esl and isl dictionary keys are 'formetric'. The italian dataset has both.


In [ ]:
print("Balgrist ESL: ", np.asarray(metadata_bal["isl"]["pcdicomapp"]).shape)
print("Croatian ESL: ", np.asarray(metadata_cro["isl"]["formetric"]).shape)
print("Italian pcdicomapp ESL: ", np.asarray(metadata_it["isl"]["pcdicomapp"]).shape)
print("Italian ESL formetric: ", np.asarray(metadata_it["isl"]["formetric"]).shape)
print("UKBB ESL:", np.asarray(metadata_ukbb["isl"]["pcdicomapp"]).shape)

print("Balgrist ISL: ", np.asarray(metadata_bal["esl"]["pcdicomapp"]).shape)
print("Croatian ISL: ", np.asarray(metadata_cro["esl"]["formetric"]).shape)
print("Italian pcdicomapp ISL: ", np.asarray(metadata_it["esl"]["pcdicomapp"]).shape)
print("Italian formetric ISL: ", np.asarray(metadata_it["esl"]["formetric"]).shape)
print("UKBB ISL:", np.asarray(metadata_ukbb["esl"]["pcdicomapp"]).shape)

In [ ]:
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(metadata_it["isl"]["pcdicomapp"])

o3d.visualization.draw_geometries([pcd])

## 5. Explore backscans

In [ ]:
print("Balgrist backscan: ", backscan_bal)
print("Croatian backscan: ", backscan_cro)
print("Italian backscan: ", backscan_it)
print("UKBB backscan: ", backscan_ukbb)

In [ ]:
processed_dirs = dirs_df[dirs_df["processed"] == True]

datasets = ["croatian", "italian", "balgrist", "ukbb"]

sampled_dirs = []

for dataset in datasets:
    dataset_dirs = processed_dirs[processed_dirs["backscan"].str.contains(dataset)]
    sampled_dirs += list(dataset_dirs.sample(4)["backscan"])

# load the backscan data
backscans = [o3d.io.read_point_cloud(backscan) for backscan in sampled_dirs]

# load the metadata
metadatas = [
    json.load(
        open(backscan.replace("ply", "json").replace("processed", "metadata_processed"))
    )
    for backscan in sampled_dirs
]

In [ ]:
# plot the backscans in 3d on a 4x4 grid. Use the same scale for all axes

dataset_titles = ["Nado", "IRCCS", "UKBB", "Balgrist"]

fig = plt.figure(figsize=(12, 12))

for i, backscan in enumerate(backscans):
    ax = fig.add_subplot(4, 4, i + 1, projection="3d")

    # color the points according to their z coordinate
    c = np.asarray(backscan.points)[:, 2]
    ax.scatter(
        np.asarray(backscan.points)[:, 0],
        np.asarray(backscan.points)[:, 2],
        np.asarray(backscan.points)[:, 1],
        c=c,
        cmap="viridis",
        s=0.1,
    )

    # set lims to max coordinate of the backscan in any axis

    lim_max, lim_min = np.max(np.abs(np.asarray(backscan.points))), -np.max(
        np.abs(np.asarray(backscan.points))
    )
    ax.set_xlim(lim_min, lim_max)
    ax.set_ylim(lim_min, lim_max)
    ax.set_zlim(lim_min, lim_max)

    # switch axis direction from negative to positive to positive to negative
    ax.invert_yaxis()

    # remove grid, add axis labels in mm, add a title to each row with dataset
    ax.set_xlabel("X (mm)")
    ax.set_ylabel("Z (mm)")
    ax.set_zlabel("Y (mm)")

    if i % 4 == 2:
        ax.annotate(
            dataset_titles[i // 4],
            xy=(-0.15, 1.05),
            xycoords="axes fraction",
            ha="center",
            fontsize=12,
            fontfamily="serif",
        )

    plt.tight_layout()

In [ ]:
from spinediffusion.datamodule.transforms.projecting import ProjectToPlane

projector = ProjectToPlane(128, 128, 0.9, 1, "mean")

# project the backscans to the sagittal plane
backscans_proj = []
for backscan, metadata in zip(backscans, metadatas):
    data_id = {}
    data_id["backscan"] = backscan
    special_points = metadata["specialPts"]
    data_id["special_points"] = {k: np.array(v) for k, v in special_points.items()}
    backscans_proj.append(projector(data_id)["depth_map"])

# plot the projected backscans in 2d on a 4x4 grid

fig = plt.figure(figsize=(12, 12))

for i, backscan_proj in enumerate(backscans_proj):
    ax = fig.add_subplot(4, 4, i + 1)
    ax.imshow(backscan_proj.squeeze(), cmap="gray")
    ax.axis("off")

    if i % 4 == 2:
        ax.annotate(
            dataset_titles[i // 4],
            xy=(-0.15, 1.05),
            xycoords="axes fraction",
            ha="center",
            fontsize=12,
            fontfamily="serif",
        )

## 6. Explore whole dataset

In [ ]:
processed_dirs_df = dirs_df[dirs_df["processed"] == True]

metadata = {}
backscans = {}

for backscan_dir, metadata_dir in tqdm(
    zip(processed_dirs_df["backscan"], processed_dirs_df["metadata"]),
    total=len(processed_dirs_df),
):
    with open(metadata_dir, "r") as f:
        metadata_ = json.load(f)

    unique_id = f"{metadata_['dataset']}_{metadata_['id']}"

    metadata[unique_id] = metadata_
    backscans[unique_id] = o3d.io.read_point_cloud(backscan_dir)

In [ ]:
backscans_dataset = {
    "balgrist": [],
    "croatian": [],
    "italian": [],
    "ukbb": [],
}

esl_dataset = {
    "balgrist": [],
    "croatian": [],
    "italian": [],
    "ukbb": [],
}

isl_dataset = {
    "balgrist": [],
    "croatian": [],
    "italian": [],
    "ukbb": [],
}

for unique_id in metadata:
    dataset = metadata[unique_id]["dataset"]
    esl = metadata[unique_id]["esl"]
    isl = metadata[unique_id]["isl"]

    backscans_dataset[dataset].append(backscans[unique_id])
    esl_dataset[dataset].append(esl)
    isl_dataset[dataset].append(isl)

### Visualize point distributions

In [ ]:
for dataset in backscans_dataset:
    points = np.concatenate(
        [np.asarray(pc.points) for pc in backscans_dataset[dataset]]
    )
    x, y, z = points.T

    fig, axs = plt.subplots(1, 3, figsize=(24, 8))

    axs[0].hist(x, bins=100, color="r", alpha=0.7)
    axs[0].set_title(f"{dataset} x-axis")
    axs[0].set_xlabel("x-axis")
    axs[0].set_ylabel("Frequency")

    axs[1].hist(y, bins=100, color="g", alpha=0.7)
    axs[1].set_title(f"{dataset} y-axis")
    axs[1].set_xlabel("y-axis")
    axs[1].set_ylabel("Frequency")

    axs[2].hist(z, bins=100, color="b", alpha=0.7)
    axs[2].set_title(f"{dataset} z-axis")
    axs[2].set_xlabel("z-axis")
    axs[2].set_ylabel("Frequency")

    plt.show()

In [ ]:
for dataset in esl_dataset:
    key = "pcdicomapp"
    if dataset == "croatian":
        key = "formetric"
    points = np.concatenate([np.asarray(pc[key]) for pc in esl_dataset[dataset]])
    x, y, z = points.T

    fig, axs = plt.subplots(1, 3, figsize=(24, 8))

    axs[0].hist(x, bins=100, color="r", alpha=0.7)
    axs[0].set_title(f"{dataset} x-axis")
    axs[0].set_xlabel("x-axis")
    axs[0].set_ylabel("Frequency")

    axs[1].hist(y, bins=100, color="g", alpha=0.7)
    axs[1].set_title(f"{dataset} y-axis")
    axs[1].set_xlabel("y-axis")
    axs[1].set_ylabel("Frequency")

    axs[2].hist(z, bins=100, color="b", alpha=0.7)
    axs[2].set_title(f"{dataset} z-axis")
    axs[2].set_xlabel("z-axis")
    axs[2].set_ylabel("Frequency")

    plt.show()

In [ ]:
for dataset in esl_dataset:
    key = "pcdicomapp"
    if dataset == "croatian":
        key = "formetric"
    points = np.concatenate([np.asarray(pc[key]) for pc in esl_dataset[dataset]])
    x, y, z = points.T

    fig, axs = plt.subplots(1, 3, figsize=(24, 8))

    axs[0].hist(x, bins=100, color="r", alpha=0.7)
    axs[0].set_title(f"{dataset} x-axis")
    axs[0].set_xlabel("x-axis")
    axs[0].set_ylabel("Frequency")

    axs[1].hist(y, bins=100, color="g", alpha=0.7)
    axs[1].set_title(f"{dataset} y-axis")
    axs[1].set_xlabel("y-axis")
    axs[1].set_ylabel("Frequency")

    axs[2].hist(z, bins=100, color="b", alpha=0.7)
    axs[2].set_title(f"{dataset} z-axis")
    axs[2].set_xlabel("z-axis")
    axs[2].set_ylabel("Frequency")

    plt.show()

In [ ]:
for unique_id in metadata:
    fix_points = np.asarray(metadata[unique_id]["fix_points"])

In [ ]:
metadata["balgrist_10"]["specialPts"]

In [ ]:
metadata["balgrist_1"]["specialPts"]

### Get common pipelineSteps

In [ ]:
pipeline_steps = []
order = []

for unique_id in metadata:
    pipeline_steps.append(metadata[unique_id]["pipelineSteps"])
    order.append(metadata[unique_id]["pipelineSteps"]["order"])

In [ ]:
set(order[0]).intersection(*order[1:])

Pipeline steps applied to all samples (above).

In [ ]:
pipeline_steps[0]["preprocessing"]

In [ ]:
points = np.asarray(pc.points)

points = points / points.max(axis=0)

norm_pc = o3d.geometry.PointCloud(points=o3d.utility.Vector3dVector(points))

In [ ]:
o3d.visualization.draw_geometries([norm_pc])